## How to make your testing life easier by using unittest.**mock** package

### A small introduction to mocking, APIs and cats

In [101]:
# Hidden imports go here
import sys
import ipytest
import requests
from unittest import mock

Let's start with the *very* basics:

In [ ]:
def a_random_function(arg1: str, arg2: str) -> str:
    return arg2 + arg1

a_random_function("AMRO", "ABN")

Now let's now make things slightly less linear:

In [ ]:
def another_function(arg1: int, arg2: int, arg3: str, arg4: str) -> str:
    return a_random_function(arg3, arg4) + str((arg1 + arg2)//2)

another_function(2, 4, "ʘ=)∫", "(=ʘᆽ")

### How is this related to unit testing?    
Let's take the definition of *unit testing*. According to Wikipedia (the main source of information for our century), we can define unit testing as:
>"A software testing method by which individual units of source code — sets of one or more computer program modules together with associated control data, usage procedures, and operating procedures — are tested to determine whether they are fit for use."

The dilemma is, *how can we test something over which we have no control*?
In our specific case, let's take `a_random_function(arg1, arg2)` as given. Let's say we import it from some other package, that doesn't belong to us.    
After all, we run into this issue many times: the most common time is when we interact with the filesystem.

Let's redefine our `another_function()` as follows:

In [ ]:
def another_function(arg1: str, arg2: str) -> str:
    return a_random_function(sys.argv[0], sys.argv[1]) + " - " + str((arg1 + arg2)//2)

another_function(1, 2)

As you can see, I have no possible control over `sys.argv`: they get defined at runtime, once I run my application.    
What is then the best way of testing for them?

Fortunately, `unittest.mock` allows us to perform this task in a relatively relaxed way.

In [99]:
@mock.patch("sys.argv", ["One", "Two"])
def another_function(arg1: str, arg2: str) -> str:
    return a_random_function(sys.argv[0], sys.argv[1]) + " - " + str((arg1 + arg2)//2)

another_function(10, 200)

'TwoOne - 105'

In [98]:
print(sys.argv)

['/home/gian/.pyenv/versions/3.7.0/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/home/gian/.local/share/jupyter/runtime/kernel-2bd513ec-9748-4168-b513-07ad5a95e220.json']


![cat](http://localhost:5500/lib/img1.jpg)

In [ ]:
from neon.functions import *
ipytest.autoconfig()
test_args = ["--showlocals", 
            "-x", 
            "--cov-report", 
            "term-missing",
            "--cov",
            "neon.functions"]

## Let's get to our cats
Since I really love cats, I'd like to know more about them. Fortunately, someone created the **completely free** [Cat facts API](https://catfact.ninja/) which can return nice (and interesting) facts about our feline friends.

In [108]:
requests.get(url="https://catfact.ninja/fact").json()["fact"]

'Mohammed loved cats and reportedly his favorite cat, Muezza, was a tabby. Legend says that tabby cats have an “M” for Mohammed on top of their heads because Mohammad would often rest his hand on the cat’s head.'

In [ ]:
class TestAPI(unittest.TestCase):
    @classmethod
    def setUpClass(cls) -> None:
        cls.mocked_data : List[dict] = [{"fact" : "This is a random fact", "length" : "-99"}]

    @mock.patch("neon.functions.make_request", return_value=None)
    def test_retrieve_data_without_waiting(self, patched_request):
        actual: dict = retrieve_data(waiting=5)
        actual_keys: list = [key for key in actual]
        expected_keys: list = ["fact", "length"]
        self.assertEqual(actual_keys, expected_keys)
    
    @mock.patch("neon.functions.retrieve_data")
    def test_process_data_with_random_load(self, patched_retrieve):
        
        actual = process_data(usernumber=5, waiting=1)
        pass

ipytest.run(*test_args)

In [ ]:
class TestSparkFunctions(unittest.TestCase):
    """
    All tests for Spark-related functions go here.
    """
    def test_group_and_save_with_random_load(self):
        pass

    def test_group_and_save_with_api_load(self):
        pass

ipytest.run(*test_args)

### Useful links:
[Catfacts (API reference)](https://catfact.ninja/)
[This application (Git repo)](https://github.com/jean-n92/pytest-mock-presentation)
[Where to patch (Documentation)](https://docs.python.org/3/library/unittest.mock.html#where-to-patch)